In [10]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

In [2]:
def xavier_init(size):
    in_dim = size[0]
    xavier_stddev = 1. / tf.sqrt(in_dim / 2.)
    return tf.random_normal(shape=size, stddev=xavier_stddev)

In [36]:
X = tf.placeholder(tf.float32, shape=[None, 784])
D_W1 = tf.Variable(xavier_init([784, 128]))
D_b1 = tf.Variable(tf.zeros(shape=[128]))
D_W2 = tf.Variable(xavier_init([128, 1]))
D_b2 = tf.Variable(tf.zeros(shape=[1]))
D_params = [D_W1, D_b1, D_W2, D_b2]

In [37]:
# Z is noised input
# G(z) => generated new x
Z = tf.placeholder(tf.float32, shape=[None, 100])
G_W1 = tf.Variable(xavier_init([100, 128]))
G_b1 = tf.Variable(tf.zeros(shape=[128]))
G_W2 = tf.Variable(xavier_init([128, 784]))
G_b2 = tf.Variable(tf.zeros(shape=[784]))
G_params = [G_W1, G_b1, G_W2, G_b2]

In [38]:
def discriminator(x):
    D_a1 = tf.matmul(x, D_W1) + D_b1
    D_h1 = tf.nn.relu(D_a1)
    # what is logit?
    # logit is inverse function of sigmoid function
    # therefore, D_logit = logit(D_prob).
    # 즉, sigmoid 함수에 들어가는 값이 logit 이라고 할 수 있음.
    D_logit = tf.matmul(D_h1, D_W2) + D_b2
    D_prob = tf.nn.sigmoid(D_logit)
    
    return D_prob, D_logit

In [39]:
def generator(z):
    G_a1 = tf.matmul(z, G_W1) + G_b1
    G_h1 = tf.nn.relu(G_a1)
    # what is differ between logit <> log_prob ?
    # 이것도 걍 로짓이 맞는거 같은데? logit_prob 의 약자일것 같음.
    G_log_prob = tf.matmul(G_h1, G_W2) + G_b2
    G_prob = tf.nn.sigmoid(G_log_prob)
    
    return G_prob

In [40]:
def sample_Z(m, n):
    return np.random.uniform(-1., 1., size=[m, n])

In [41]:
def plot(samples):
    fig = plt.figure(figsize=(4, 4))
    gs = gridspec.GridSpec(4, 4)
    gs.update(wspace=0.05, hspace=0.05)

    for i, sample in enumerate(samples):
        ax = plt.subplot(gs[i])
        plt.axis('off')
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        ax.set_aspect('equal')
        plt.imshow(sample.reshape(28, 28), cmap='Greys_r')

    return fig

In [46]:
G_sample = generator(Z)
D_real, D_logit_real = discriminator(X)
D_fake, D_logit_fake = discriminator(G_sample)

# V1. paper-based learning (다른 코드를 보면 logit 을 이용해서 CE로 계산을 함. 이게 더 성능이 잘 나오나봄)
# http://bamos.github.io/2016/08/09/deep-completion/ 참고

# maximize x => minimize -x
D_loss = -tf.reduce_mean(tf.log(D_real) + tf.log(1-D_fake))
G_loss = -tf.reduce_mean(tf.log(D_fake))

# V2.
# D_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_real, labels=tf.ones_like(D_logit_real)))
# D_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.zeros_like(D_logit_fake)))
# D_loss = D_loss_real + D_loss_fake
# G_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_logit_fake, labels=tf.ones_like(D_logit_fake)))

D_solver = tf.train.AdamOptimizer().minimize(D_loss, var_list=D_params)
G_solver = tf.train.AdamOptimizer().minimize(G_loss, var_list=G_params)

In [43]:
mnist = input_data.read_data_sets('MNIST_data/', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [44]:
batch_size = 128
Z_dim = 100

In [47]:
sess = tf.Session()
# sess.run(tf.initialize_all_variables())
sess.run(tf.global_variables_initializer())

idx = 0

for it in range(1000000):
    if it % 1000 == 0:
        samples = sess.run(G_sample, feed_dict={Z: sample_Z(16, Z_dim)})
        
        fig = plot(samples)
        plt.savefig('out/{}.png'.format(idx), bbox_inches='tight')
        idx += 1
        plt.close(fig)
    
    # _ 는 원래 Y 임.
    X_batch, _ = mnist.train.next_batch(batch_size)
    
    _, D_loss_curr = sess.run([D_solver, D_loss], feed_dict={X: X_batch, Z: sample_Z(batch_size, Z_dim)})
    _, G_loss_curr = sess.run([G_solver, G_loss], feed_dict={Z: sample_Z(batch_size, Z_dim)})
    
    if it % 1000 == 0:
        print('Iter: {}'.format(it))
        print('D loss: {:.4}'. format(D_loss_curr))
        print('G_loss: {:.4}'.format(G_loss_curr))
        print()

Iter: 0
D loss: 1.061
G_loss: 2.636

Iter: 1000
D loss: 0.01082
G_loss: 9.352

Iter: 2000
D loss: 0.01764
G_loss: 5.479

Iter: 3000
D loss: 0.08085
G_loss: 5.206

Iter: 4000
D loss: 0.1681
G_loss: 5.829

Iter: 5000
D loss: 0.1976
G_loss: 4.92

Iter: 6000
D loss: 0.2551
G_loss: 4.014

Iter: 7000
D loss: 0.2636
G_loss: 4.74

Iter: 8000
D loss: 0.3662
G_loss: 3.301

Iter: 9000
D loss: 0.5028
G_loss: 3.265

Iter: 10000
D loss: 0.4371
G_loss: 3.141

Iter: 11000
D loss: 0.5837
G_loss: 2.976

Iter: 12000
D loss: 0.607
G_loss: 2.534

Iter: 13000
D loss: 0.5519
G_loss: 2.497

Iter: 14000
D loss: 0.7838
G_loss: 2.087

Iter: 15000
D loss: 0.7077
G_loss: 2.191

Iter: 16000
D loss: 0.7375
G_loss: 2.321

Iter: 17000
D loss: 0.8381
G_loss: 2.239

Iter: 18000
D loss: 0.4937
G_loss: 2.174

Iter: 19000
D loss: 0.6957
G_loss: 2.227

Iter: 20000
D loss: 0.6854
G_loss: 2.39

Iter: 21000
D loss: 0.5497
G_loss: 1.933

Iter: 22000
D loss: 0.6943
G_loss: 2.055

Iter: 23000
D loss: 0.6333
G_loss: 2.513

Iter: 2

/Users/nhnent/.pyenv/versions/anaconda3-4.3.0/lib/python3.6/site-packages/numpy/ma/core.py:4185: UserWarning: Warning: converting a masked element to nan.
  warnings.warn("Warning: converting a masked element to nan.")


Iter: 81000
D loss: nan
G_loss: nan

Iter: 82000
D loss: nan
G_loss: nan

Iter: 83000
D loss: nan
G_loss: nan

Iter: 84000
D loss: nan
G_loss: nan

Iter: 85000
D loss: nan
G_loss: nan

Iter: 86000
D loss: nan
G_loss: nan

Iter: 87000
D loss: nan
G_loss: nan

Iter: 88000
D loss: nan
G_loss: nan

Iter: 89000
D loss: nan
G_loss: nan

Iter: 90000
D loss: nan
G_loss: nan

Iter: 91000
D loss: nan
G_loss: nan

Iter: 92000
D loss: nan
G_loss: nan

Iter: 93000
D loss: nan
G_loss: nan

Iter: 94000
D loss: nan
G_loss: nan

Iter: 95000
D loss: nan
G_loss: nan

Iter: 96000
D loss: nan
G_loss: nan

Iter: 97000
D loss: nan
G_loss: nan

Iter: 98000
D loss: nan
G_loss: nan

Iter: 99000
D loss: nan
G_loss: nan

Iter: 100000
D loss: nan
G_loss: nan

Iter: 101000
D loss: nan
G_loss: nan

Iter: 102000
D loss: nan
G_loss: nan

Iter: 103000
D loss: nan
G_loss: nan

Iter: 104000
D loss: nan
G_loss: nan

Iter: 105000
D loss: nan
G_loss: nan

Iter: 106000
D loss: nan
G_loss: nan

Iter: 107000
D loss: nan
G_loss

KeyboardInterrupt: 